In [100]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

#### Structured Outputs
Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema, so you don't need to worry about the model omitting a required key, or hallucinating an invalid enum value.

Some benefits of Structured Outputs include:

- Reliable type-safety: No need to validate or retry incorrectly formatted responses
- Explicit refusals: Safety-based model refusals are now programmatically detectable
- Simpler prompting: No need for strongly worded prompts to achieve consistent formatting

In [44]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    max_tokens=50,
    response_format=CalendarEvent
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Parsed Result: {completion.choices[0].message.parsed}")
print(f"Content Result: {completion.choices[0].message.content}")


print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion.usage.to_dict()}")

------------------------------
Details about response
------------------------------
Model: gpt-4o-2024-08-06


------------------------------
Details about Results
------------------------------
Parsed Result: name='Science Fair' date='Friday' participants=['Alice', 'Bob']
Content Result: {"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}


------------------------------
Details about Usage
------------------------------
Result: {'completion_tokens': 18, 'prompt_tokens': 92, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}


#### Compare the prompt usage with chat completion call

In [36]:
completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    max_tokens=50
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Result: {completion.choices[0].message.content}")

print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion.usage.to_dict()}")

------------------------------
Details about response
------------------------------
Model: gpt-4o-2024-08-06


------------------------------
Details about Results
------------------------------
Result: Event: Science fair  
Participants: Alice and Bob  
Date: Friday  


------------------------------
Details about Usage
------------------------------
Result: {'completion_tokens': 16, 'prompt_tokens': 28, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}


#### Add the extraction json schema in prompt usage with chat completion call

In [42]:
json_prompt = f"""
Extract the event information in following json schema.

{{
"name": "",
"date": "",
"participants": [""]
}}

"""


completion_json = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": f"{json_prompt}"},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    max_tokens=50
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion_json.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Result: {completion_json.choices[0].message.content}")

print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion_json.usage.to_dict()}")

------------------------------
Details about response
------------------------------
Model: gpt-4o-2024-08-06


------------------------------
Details about Results
------------------------------
Result: {
"name": "Science Fair",
"date": "Friday",
"participants": ["Alice", "Bob"]
}


------------------------------
Details about Usage
------------------------------
Result: {'completion_tokens': 24, 'prompt_tokens': 48, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}


#### Conclusion
1. Chat completion without response format uses less tokens
2. Chat completion with json response format defined in prompt uses more token that 1
3. Chat completion with response format defined uses the hhighest token

### Examples of structured output

#### 1. Chain of Thought

In [56]:
from pydantic import BaseModel

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed
math_reasoning

MathReasoning(steps=[Step(explanation="We start by isolating the term with the variable, 8x. To do this, we need to eliminate the constant term on the left side of the equation. The equation is 8x + 7 = -23. We'll subtract 7 from both sides to start the isolation of 8x.", output='8x + 7 - 7 = -23 - 7'), Step(explanation='Subtracting 7 from both sides, we eliminate the 7 on the left, which simplifies our equation to 8x on the left side.', output='8x = -30'), Step(explanation="Next, we need to solve for x. This means we need x to stand alone on one side of the equation. 8x means 8 multiplied by x, so we'll divide both sides of the equation by 8 to isolate x.", output='x = -30 / 8'), Step(explanation='Simplifying the fraction -30/8 by dividing the numerator and the denominator by their greatest common divisor, which is 2, we get -15/4.', output='x = -15/4')], final_answer='x = -15/4')

#### 2. Structured Data Extraction

In [63]:
from pydantic import BaseModel

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": "Bhagvad gita"}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed
research_paper

ResearchPaperExtraction(title='The Role of Bhagavad Gita in Modern Life', authors=['Ravi Shankar', 'Leela Menon'], abstract="The Bhagavad Gita, a 700-verse Hindu scripture, is part of the Indian epic Mahabharata. It is a conversation between prince Arjuna and the god Krishna, who serves as his charioteer. This discourse, which takes place on the battlefield of Kurukshetra, addresses the moral and philosophical dilemmas faced by Arjuna. In this paper, we explore the Gita's applicability to modern life, focusing on its teachings about duty, righteousness, and devotion. We analyze how the Gita's insights can be integrated into contemporary ethical and spiritual discussions, offering timeless guidance for dealing with today's challenges.", keywords=['Bhagavad Gita', 'philosophy', 'ethics', 'spirituality', 'modern life'])

### Handle the edge case
In some cases, the model might not generate a valid response that matches the provided JSON schema.

This can happen in the case of a `refusal`, if the model refuses to answer for safety reasons, or if for example you reach a max tokens limit and the response is incomplete.

In [77]:
from pydantic import BaseModel

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Do your thing"},
        {"role": "user", "content": ""}
    ],
    response_format=ResearchPaperExtraction,
)

message = completion.choices[0].message

# If the model refuses to respond, you will get a refusal message
if (message.refusal):
  print(math_reasoning.refusal)
else:
  research_paper = message.parsed

research_paper

ResearchPaperExtraction(title='Chatbots in Mental Health: Bridging the Gap between Technology and Therapy', authors=['Emily R. Watson', 'James L. Nguyen', 'Sophia M. Patel'], abstract='The integration of chatbot technology in mental health services presents a novel approach to enhancing patient engagement and accessibility. This paper explores the potential of chatbots as a supplementary tool for mental health support, examining their benefits, limitations, and the ethical considerations involved. Through a review of existing literature and analysis of current chatbot applications, we identify key areas where chatbots can contribute to mental health treatment, especially in resource-limited environments. Our findings suggest that, while chatbots cannot replace traditional therapy models, they can effectively complement them by offering continuous support and assisting therapists in tracking patient progress.', keywords=['chatbot', 'mental health', 'therapy', 'artificial intelligence', 

#### Structured output can also be obtained using chat.completion.create functions

```
response_format: { "type": "json_schema", "json_schema": … , "strict": true }
```

In [ ]:
try:
    response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object", # Root(First) should be object
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"], # Define the properties which is required
                "additionalProperties": False # controls whether it is allowable for an object to contain additional keys / values that were not defined in the JSON Schema.
            },
        "strict": True
        }
    }
    )
except Exception as e:
    response = None
    print(e)

if response:
    content = response.choices[0].message.content
    print(content)


{"steps":[{"explanation":"The equation is 8x + 7 = -23. The goal is to solve for x. The first step is to isolate the term containing x on one side of the equation. To do this, subtract 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides of the equation. On the left side, 7 - 7 cancels out, and on the right side, we calculate -23 - 7.","output":"8x = -30"},{"explanation":"Now, the equation is 8x = -30. To solve for x, divide both sides of the equation by 8, which is the coefficient of x.","output":"x = -30 / 8"},{"explanation":"Simplify -30 / 8. The greatest common divisor of 30 and 8 is 2. Divide both the numerator and the denominator by 2 to simplify the fraction.","output":"x = -15 / 4"},{"explanation":"Thus, the solution to the equation 8x + 7 = -23 is x = -15/4.","output":"x = -15/4"}],"final_answer":"x = -15/4"}


#### Validate the schema of content


In [ ]:

from pydantic import BaseModel, ValidationError
from typing import List

# Define types that match the JSON Schema using pydantic models
class Step(BaseModel):
  explanation: str
  output: str

class Solution(BaseModel):
  steps: List[Step]
  final_answer: str


try:
  # Parse and validate the response content
  solution = Solution.model_validate_json(content)
  print(solution)
except ValidationError as e:
  # Handle validation errors
  print(e.json())



#### Supported Schemas
- String
- Number
- Boolean
- Integer
- Object
- Array
- Enum
- anyOf

Note: The root level object of a schema must be an `object`, and not use `anyOf`